# Readout weight calibration

In this notebook, you will learn how to calibrate and use optimal integration weights to distinguish between qubits states in circuit QED.

This demonstration runs without connection to real qubits, assuming a loopback on the readout drive line directly into the readoud acquisition line. We emulate the measurement signals corresponding to different qubit states by two different measurement pulses, differing only by a phase.

## 0. General Imports and Definitions

### 0.1 Python Imports 

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from laboneq.analysis import calculate_integration_kernels_thresholds

# Helpers:
from laboneq.contrib.example_helpers.feedback_helper import (
    create_calibration_experiment,
    create_integration_verification_experiment,
    state_emulation_pulse,
)
from laboneq.contrib.example_helpers.generate_device_setup import (
    generate_device_setup_qubits,
)

# all LabOne Q functionality
from laboneq.simple import *

## 1. Device setup and calibration

### 1.1 Generate a calibrated Device Setup and qubit objects

We'll generate a device setup and some qubit objects using a set of pre-defined parameters in a helper function. 

In [ ]:
# specify the number of qubits you want to use
number_of_qubits = 2

# generate the device setup and the qubit objects using a helper function
device_setup, qubits = generate_device_setup_qubits(
    number_qubits=number_of_qubits,
    pqsc=[{"serial": "DEV10001"}],
    hdawg=[
        {
            "serial": "DEV8001",
            "number_of_channels": 8,
            "options": None,
        }
    ],
    shfqc=[
        {
            "serial": "DEV12001",
            "number_of_channels": 6,
            "readout_multiplex": 6,
            "options": None,
        }
    ],
    multiplex_drive_lines=True,
    include_flux_lines=True,
    server_host="localhost",
    setup_name=f"my_{number_of_qubits}_tunable_qubit_setup",
)

q0, q1 = qubits[:2]

### 1.2 Adapt setup calibration

In this notebook we are using a pulse played from a second measure line to emulate the qubit being in the excited state. In this case we want to have the same instrument settings for the two used measurement lines. 
Additionally, for the method of readout weight calibration demonstrated in this notebook, the acquire line should not be modulated, as the calculated readout weights already contain the software modulation by construction.

In [ ]:
readout_weight_calibration = Calibration()
readout_weight_calibration["/logical_signal_groups/q1/measure"] = (
    device_setup.get_calibration()["/logical_signal_groups/q0/measure"]
)
readout_weight_calibration["/logical_signal_groups/q0/acquire"] = (
    device_setup.get_calibration()["/logical_signal_groups/q0/acquire"]
)
readout_weight_calibration["/logical_signal_groups/q0/acquire"].oscillator = None

# print(readout_weight_calibration)

device_setup.set_calibration(readout_weight_calibration)

# print(device_setup.get_calibration())

In [ ]:
# use emulation mode - no connection to instruments
use_emulation = True

# create and connect to a session
session = Session(device_setup=device_setup)
session.connect(do_emulation=use_emulation)

## 2. Calibration of state discrimination

We determine the optimal integration weights by measuring traces of the qubit states and computing an integration kernel using the toolkit routines. We simulate different qubit responses by playing pulses with different phases and amplitudes on the readout line. We have to make sure that the traces are a multiple of 16 samples long.

### 2.1 Obtain traces

In [ ]:
num_states = 2
used_qubits = [q0, q1]

calibration_experiment = create_calibration_experiment(
    state_emulation_pulse=state_emulation_pulse(),
    qubit_states=range(num_states),
    measure_signals=[q.signals["measure"] for q in used_qubits],
    acquire_signal=q0.signals["acquire"],
)

calibration_results = session.run(calibration_experiment)

calibration_traces = []
for it in range(num_states):
    trace = calibration_results.get_data(f"raw_{it}")
    calibration_traces.append(trace[: (len(trace) // 16) * 16])


# In emulation mode, the 'acquired' traces are all identical. Consequently, the computation of the optimal
# discrimination weights will fail. Instead we 'patch' the traces with an artificial phase.
if use_emulation:
    for i in range(num_states):
        phase = np.exp(2j * np.pi * i / num_states)
        calibration_traces[i] *= phase

### 2.2 Compute kernels

We only need the number of states minus 1 kernels, the additional kernel is computed on the device.

In [ ]:
# Calculate and plot kernels
kernels, thresholds = calculate_integration_kernels_thresholds(calibration_traces)

for i, k in enumerate(kernels):
    plt.plot(k.samples.real, k.samples.imag, ["ro-", "gx-", "b+-"][i], alpha=0.2)

### 2.3 Plot acquired results after readout optimization

When using the optimized kernels calculated in the last step, the integration results for ground and excited states are rotated so that a projection onto the real axis allows for later discrimination with a real-valued threshold.

In [ ]:
my_exp = create_integration_verification_experiment(
    measure_lines=[q0.signals["measure"], q1.signals["measure"]],
    acquire_line=q0.signals["acquire"],
    kernels=kernels,
    state_emulation_pulse=state_emulation_pulse,
    thresholds=thresholds,
)

integration_results = session.run(my_exp)

In [ ]:
s0 = integration_results.get_data("data_0").real
s1 = integration_results.get_data("data_1").real

plt.plot(s0, ".b")
plt.plot(s1, ".r")